In [ ]:
# Install required libraries
!pip install geopandas plotly pandas

# Import necessary libraries
import pandas as pd
import geopandas as gpd
import plotly.express as px

# File paths
eci_data_path = "multidimensional_eci_data.csv"
trade_data_path = "Export COMPET_.csv"
exposure_data_path = "Fig2a-avg_exposure.csv"  # Newly added exposure data
shapefile_path = "ne_110m_admin_0_countries.shp"

# Step 1: Load datasets
eci_data = pd.read_csv(eci_data_path)
trade_data = pd.read_csv(trade_data_path)
exposure_data = pd.read_csv(exposure_data_path)  # Load exposure data
world = gpd.read_file(shapefile_path)

# Step 2: Process ECI data for 2019
eci_2019 = eci_data[eci_data['variable'] == 'eci_trade'][['country', 'x2019']].rename(columns={'x2019': 'eci_trade'})
eci_2019 = eci_2019.dropna()

# Step 3: Filter trade data for selected indicators
quantity_indicator = "Adjusted export market share - Quantity (delta log)"
quarters_2018 = ['2018q1', '2018q2', '2018q3', '2018q4']

trade_data = trade_data[
    (trade_data['Indicator'] == quantity_indicator) &
    (trade_data['Partner'] == 'World') &
    (trade_data['Attribute 1'] == 'All')
].copy()

for col in quarters_2018:
    trade_data[col] = pd.to_numeric(trade_data[col], errors='coerce')

trade_data['avg_quantity_2018'] = trade_data[quarters_2018].mean(axis=1)
trade_agg = trade_data.groupby('Economy ISO3')['avg_quantity_2018'].mean().reset_index()
trade_agg = trade_agg.rename(columns={'Economy ISO3': 'country', 'avg_quantity_2018': 'quantity_market_share'})

# Step 4: Process exposure data (country-specific self-exposure)
exposure_data.set_index('Unnamed: 0', inplace=True)  # Set country as the index
self_exposure = exposure_data.stack().reset_index()
self_exposure = self_exposure[self_exposure['Unnamed: 0'] == self_exposure['level_1']]  # Select diagonal elements
self_exposure = self_exposure.rename(columns={'Unnamed: 0': 'country', 0: 'self_exposure'})[['country', 'self_exposure']]

# Step 5: Merge datasets
merged_data = eci_2019.merge(trade_agg, on='country', how='left')
merged_data = merged_data.merge(self_exposure, on='country', how='left')

# Step 6: Merge with shapefile data for mapping
world = world.merge(merged_data, left_on='ADM0_A3', right_on='country', how='left')

# Step 7: Create the choropleth map with hover text
fig = px.choropleth(
    world,
    geojson=world.geometry,
    locations=world.index,  # Use GeoPandas indices for location mapping
    color='eci_trade',  # Color represents ECI
    color_continuous_scale='Viridis',
    title='Economic Complexity Index (ECI) and Trade Indicators by Country',
    labels={'eci_trade': 'ECI'},
    hover_name='ADM0_A3',  # Country code in hover text
    hover_data={
        'eci_trade': True,                 # Display ECI
        'quantity_market_share': ':.4f',  # Display adjusted export market share
        'self_exposure': ':.4f'           # Display country-specific shock exposure
    }
)

# Step 8: Update layout for better visuals
fig.update_geos(
    showcountries=True,
    showcoastlines=True,
    fitbounds="locations",
    visible=False
)
fig.update_layout(
    margin={"r": 0, "t": 30, "l": 0, "b": 0},
    coloraxis_colorbar=dict(title="ECI")
)

#adding part
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(label="ECI",
                     method="update",
                     args=[{"z": [world['eci_trade']]}]),
                dict(label="Trade Value",
                     method="update",
                     args=[{"z": [world['quantity_market_share']]}]),
                dict(label="Exposure",
                     method="update",
                     args=[{"z": [world['self_exposure']]}])
            ],
            direction="down",
            showactive=True
        )
    ]
)

#heatmap

import seaborn as sns
import matplotlib.pyplot as plt

# Correlation heatmap
corr_data = world[['eci_trade', 'quantity_market_share', 'self_exposure']].dropna()
corr_matrix = corr_data.corr()

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# Add annotations for the top 3 countries by ECI
top_countries = world.nlargest(3, 'eci_trade')

for _, row in top_countries.iterrows():
    fig.add_annotation(
        x=row.geometry.centroid.x,
        y=row.geometry.centroid.y,
        text=f"{row['ADM0_A3']}: ECI {row['eci_trade']:.2f}",
        showarrow=True,
        arrowhead=2
    )

#bubble
import numpy as np
import plotly.graph_objects as go

# Ensure valid data for marker size
world['quantity_market_share_clean'] = world['quantity_market_share'].replace([np.inf, -np.inf], np.nan)  # Remove infinities
world['quantity_market_share_clean'] = world['quantity_market_share_clean'].fillna(0)  # Replace NaNs with 0
world['quantity_market_share_clean'] = world['quantity_market_share_clean'].apply(lambda x: max(x, 0))  # Ensure no negative values

# Add bubble markers for trade values
fig.add_trace(go.Scattergeo(
    lon=world.geometry.centroid.x,
    lat=world.geometry.centroid.y,
    mode='markers',
    marker=dict(
        size=world['quantity_market_share_clean'] * 10,  # Scale trade values (adjust multiplier for better visualization)
        color=world['self_exposure'],                   # Color by exposure
        colorscale='Reds',
        showscale=True,
        colorbar=dict(title="Self-Exposure"),
        opacity=0.8,
        line=dict(width=1, color='black')
    ),
    text=world.apply(
        lambda row: f"Country: {row['ADM0_A3']}<br>Trade Value: {row['quantity_market_share_clean']:.4f}<br>Exposure: {row['self_exposure']:.4f}",
        axis=1
    ),
    hoverinfo='text'
))

#Fix for label
# Update layout for better visual organization
fig.update_layout(
    title='Economic Complexity Index (ECI) with Trade and Exposure Metrics',
    margin={"r": 0, "t": 50, "l": 0, "b": 0},
    geo=dict(
        showcountries=True,
        showcoastlines=True,
        fitbounds="locations"
    ),
    coloraxis_colorbar=dict(
        title="ECI",
        thickness=15,
        x=0.85,  # Position the ECI color bar closer to the right edge
        y=0.5
    )
)

# Add a separate legend for bubble marker colors (self-exposure)
fig.update_layout(
    legend=dict(
        title=dict(text="Exposure Levels"),
        yanchor="top",
        y=0.95,
        xanchor="right",
        x=0.95
    )
)

# Add annotation or adjust marker size legend for better distinction
fig.add_trace(go.Scattergeo(
    lon=[],
    lat=[],
    mode='markers',
    marker=dict(
        size=[10, 20, 30],  # Sample sizes for legend
        color='red',
        showscale=False,
        opacity=0.8,
        line=dict(width=1, color='black')
    ),
    showlegend=True,
    name='Trade Value (scaled)'
))


# Step 9: Display the map
fig.show()